In [ ]:
import numpy as np
from PIL import Image
import math
from numpy import pi, sin, cos, tan
import numba
from numba import njit, prange

In [ ]:
@njit
def degrees_to_rad(degrees):
    return degrees/180 * np.pi

@njit
def rad_to_degrees(rad):
    return rad/ np.pi * 180

In [ ]:
@njit
def vector_magnitude(vector):
    return np.sqrt(vector.real ** 2 + vector.imag ** 2)

In [ ]:
@njit
def dpi_to_dpmm(dpis: int):
    mm_in_a_inch = 25.4
    return dpis / mm_in_a_inch

In [ ]:
@njit
def angle_from_coordinates(x, y):
    magnitude = vector_magnitude(x, y)
    vector = complex(x, y) / magnitude
    angle = np.log(vector).imag
    return angle

In [ ]:
@njit
def vector_from_coordinates(x, y):
    vector = complex(x, y)
    return vector

In [ ]:
@njit
def unit_vector(vector):
    magnitude = vector_magnitude(vector)
    u_vector = vector / magnitude
    return u_vector

In [ ]:
@njit
def vector_to_normalized_projection_magnitude(vector, focal_distance, dpis):
    """Calculates the projection of a pixel.
    
    With the projection of a pixel, you can use an inverse_mapping_function to get the lens angle of it
    The projection is 'normalized' using the focal distance as the factor, therefore, 'normalized' may be a 
    misnomer in case of a  any mapping function that produces values bigger than 1.
    """
    dpmm = dpi_to_dpmm(dpis)
    opposite_side = vector_magnitude(vector)
    adjacent_side = focal_distance * dpmm
    
    normalized_projection_magnitude = np.absolute(opposite_side / adjacent_side)
    
    return normalized_projection_magnitude

### Mapping functions and inverses

In [ ]:
@njit
def rectilinear_inverse(normalized_projection_magnitude):
    angle = np.arctan(normalized_projection_magnitude)
    return angle

@njit
def rectilinear(teta):
    return np.tan(teta)

In [ ]:
@njit
def stereografic_inverse(normalized_projection_magnitude):
    tan_Td2 = normalized_projection_magnitude / 2
    angle_d2 = np.arctan(tan_Td2)
    angle = 2 * angle_d2
    return angle

@njit
def stereografic(teta):
    Td2 = teta / 2
    pre_projection = np.tan(Td2)
    projection = 2 * pre_projection
    return projection

In [ ]:
@njit
def equidistant_inverse(normalized_projectin_magnitude):
    return normalized_projectin_magnitude

@njit
def equidistant(teta):
    return teta

In [ ]:
@njit
def equisolid_inverse(normalized_projection_magnitude):
    sin_Td2 = normalized_projection_magnitude / 2
    angle_d2 = np.arcsin(sin_Td2)
    angle = 2 * angle_d2
    return angle

@njit
def equisolid(teta):
    teta_d2 = teta / 2
    pre_projection = np.sin(teta_d2)
    projection = 2 * pre_projection
    return projection

In [ ]:
@njit
def orthographic_inverse(normalized_projection_magnitude):
    angle = np.arcsin(normalized_projection_magnitude)
    return angle

@njit
def orthographic(teta):
    angle = np.sin(teta)
    return angle

### Helper functions

In [ ]:
# Preciso ajustar essa função para ela levar em consideração a diferença da distancia focal
@njit
def center_pixel_angle_resolution(origin_function, destiny_function):
    origin_pixel = origin_function(degrees_to_rad(1))
    destiny_pixel = destiny_function(degrees_to_rad(1))
    dpi_factor = origin_pixel / destiny_pixel
    dpi_factor = np.round(dpi_factor)
    return dpi_factor

In [ ]:
@njit
def c_round(complex_number):
    return int(np.round(complex_number.real)), int(np.round(complex_number.imag))

In [ ]:
@njit
def compute_lens_angle(vector, f_distance, dpis, inverse_function):
    normalized_magnitude = vector_to_normalized_projection_magnitude(vector, f_distance, dpis)
    angle = inverse_function(normalized_magnitude)
    return angle * 2

In [ ]:
@njit
def compute_dpis(vector, angle, f_distance, function):
    half_angle = angle / 2
    quasi_magnitude = function(half_angle) * f_distance
    v_magnitude = vector_magnitude(vector)
    dpmm = v_magnitude / quasi_magnitude
    dpis = dpmm * 25.4
    return dpis

In [ ]:
compute_dpis(complex(1440, 900), degrees_to_rad(160), 50, stereografic)

In [ ]:
rad_to_degrees(compute_lens_angle(complex(1440, 900), 50, 671, equisolid_inverse))

### Main function

In [ ]:
@njit
def process_image(o_image_arr, lens_angle, function_o, ifunction_o, function_d, ifunction_d, fullframe):
    
    # Some defaults
    f_distance = 50
    
    
    # calculate the correct destiny size
    y_size_o, x_size_o = o_image_arr.shape[:2]
    
    # define the origin center as complex number to make it easier to do the algebra later 
    center_o = complex(x_size_o, y_size_o) / 2 - complex(0.5, 0.5)
    
    # compute the dpis
    dpis = compute_dpis(center_o, lens_angle, f_distance, function_o)
    dpmm = dpi_to_dpmm(dpis)
    
    # Calculate the max angle of the lens
    if fullframe:
        max_vector_o = center_o
    else:
        max_vector_o = complex(center_o.real, 0)
        
    
    # Compute the focal distance of the destiny lens so it can have the same angle
    f_factor = vector_magnitude(max_vector_o) / (function_d(lens_angle / 2) * f_distance * dpmm)
    f_distance_d = f_distance * f_factor
    
    
    """
    # TODO Crop the black borders
    # TODO Fix the image size to retain maximum information
    
    y_size_d = y_size_o * dpi_factor
    x_size_d = x_size_o * dpi_factor
    y_size_d = int(np.round(y_size_d))
    x_size_d = int(np.round(x_size_d))
    """
    
    # Create the destiny array
    
    x_size_d = x_size_o
    y_size_d = y_size_o
    center_d = complex(x_size_d, y_size_d) / 2 - complex(0.5, 0.5)
    
    dest_arr = np.zeros((y_size_d, x_size_d, 3), 'uint8')
    
        
    """
    Firstly, the program reads the position from the destination, calculates the teta angle of the destination
    and then it translates it to a position on the origin, reads such position and writes it on the position
    from the destination
    """
    for row in prange(y_size_d):
        for column in prange(x_size_d):
            
            position_d = complex(column, row)
            position_vector = position_d - center_d
            unit_position_vector = unit_vector(position_vector)
            projection_magnitude = vector_to_normalized_projection_magnitude(position_vector, f_distance_d, dpis)
            
            teta = ifunction_d(projection_magnitude)
            
            projection_magnitude_o = function_o(teta) * f_distance * dpmm
            position_o = center_o + (unit_position_vector * projection_magnitude_o)
            
            column_o, row_o = c_round(position_o)
            
            if 0 <= column_o < x_size_o and 0 <= row_o < y_size_o:  
                dest_arr[row, column, :] = o_image_arr[row_o, column_o, :]
            
    return dest_arr

### Image Processing

In [ ]:
im = Image.open('fisheye-lens-city.jpg')

In [ ]:
temp = np.asarray(im)

In [ ]:
%timeit pi = process_image(temp, degrees_to_rad(160), equisolid, equisolid_inverse, rectilinear, rectilinear_inverse, True)

In [ ]:
Image.fromarray(pi)

In [ ]:
%timeit rpi = process_image(temp, degrees_to_rad(160), equisolid, equisolid_inverse, stereografic, stereografic_inverse, True)

In [ ]:
Image.fromarray(rpi)

# Create an array of the image and a destiny array for the adjusted image

# Calculate distances for the lens on both images

@njit(parallel=True)
def ray_adjustment(original_array, destiny_array, f_origin, f_dest):
    
    dimensions = destiny_array.shape
    row_center = (dimensions[0]-1) / 2
    col_center = (dimensions[1]-1) / 2
    
    my_dest = np.zeros(destiny_array.shape, 'uint8')
    
    original_dimensions = original_array.shape
    original_row_center = (original_dimensions[0]-1) / 2
    original_col_center = (original_dimensions[1]-1) / 2
    
    for r in prange(dimensions[0]):
        for c in range(dimensions[1]):
            row_distance = row_center - r  # rows start positive
            colum_distance = c - col_center  # columns start negative
            distance_dest = pit(row_distance, colum_distance)
            dest_lens_angle = np.arctan(distance_dest / f_dest)
            
            original_distance = 2 * np.sin(dest_lens_angle / 2) * f_origin
            
            pixel_angle_obj = (colum_distance + 1j * row_distance) / distance_dest
            
            source_row = original_row_center - ((pixel_angle_obj * original_distance).imag)
            source_column = (pixel_angle_obj * original_distance).real + original_col_center
            
            if (0 <= source_row <= original_dimensions[0]-1) and (0 <= source_column <= original_dimensions[1]-1):
                my_dest[r, c, :] = original_array[int(np.round(source_row)), int(np.round(source_column)), :]
    
    return my_dest

def adjust_image(image, lens_angle):
    original_dimensions = image.size
    
    
    # Use one half of the angle and image dimensions to calculate the focal distances in pixels
    half_dim = [x / 2 for x in original_dimensions]
    half_angle = lens_angle / 2

    f_origin = pit(*half_dim) / (2 * np.sin(half_angle / 2))
    f_dest = pit(*half_dim) / np.tan(half_angle)
    print(f_origin, f_dest)
    
    # Calculate a correction factor to not lose quality on the destiny image
    origin_degree_pixels = 2* np.sin(torad(1/2)) * f_origin
    destiny_degree_pixels = np.tan(torad(1)) * f_dest
    size_factor = origin_degree_pixels / destiny_degree_pixels
    
    
    # adjust destiny image size and focus distance
    f_dest = f_dest * size_factor
    h_dim, v_dim = np.multiply(original_dimensions, size_factor)
    
    origin_arr = np.asarray(im)
    destiny_arr = np.zeros((np.int64(v_dim), np.int64(h_dim), 3), 'uint8')  
    
    dest = ray_adjustment(origin_arr, destiny_arr, f_origin, f_dest)
    Image.fromarray(dest)
    ni = Image.fromarray(dest)
    
    return ni

A transformação é uma conversão de tangente para seno
É possivel calcular as dimensões finais da imagem com base em uma definição de angulo de abertura, calcular a distancia relativa da camera para o hemisfério de image e gerar uma nova imagem com base na projeção da tangente.

Fornecemos um angulo horizontal ng_h, e com base na proporção da imagem, calculamos o angulo horizontal ng_v.


In [ ]:
ni.save('save_test.png')